<a href="https://colab.research.google.com/github/PriyankaTUI/AudioClassificationWithDeepLearningAnalysis/blob/master/LearningWithoutForgetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/git_projects/AudioClassificationWithDeepLearningAnalysis

/content/drive/MyDrive/Colab Notebooks/git_projects/AudioClassificationWithDeepLearningAnalysis


In [3]:
!pwd

/content/drive/MyDrive/Colab Notebooks/git_projects/AudioClassificationWithDeepLearningAnalysis


In [4]:
import numpy as np
import torch
import models
import copy
import torch.nn.functional as F
from dataset import MetaAudioDataLoader

Adding google speech command dataset

In [5]:
import torchaudio

dataset = torchaudio.datasets.SPEECHCOMMANDS('./dataset/data/' , url = 'speech_commands_v0.02', folder_in_archive= 'SpeechCommands',  download = True)

split dataset and add to dataloder.

Use different transformation for it

In [ ]:
labels = sorted(list(set(datapoint[2] for datapoint in dataset)))
labels

In [ ]:
import random
random_targets = random.sample(labels, 2)
print(f"Randomly selected tagets: {random_targets}")


def label_to_index(label):
    # Return the position of the word in labels
    return torch.tensor(random_targets.index(label))


def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return random_targets[index]

tensors = []
targets = []
for waveform, _, label, *_ in dataset:
  if label in random_targets:
    if waveform.shape == (1,16000):
        tensors += [waveform]
        targets += [label_to_index(label)]


print(len(tensors))
print(len(targets))

Randomly selected tagets: ['two', 'sheila']
5279
5279


In [ ]:
targets[0]

torch.Tensor

In [ ]:
import torch.nn as nn
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.Resample(16000, 8000),
    torchaudio.transforms.MFCC(sample_rate=8000)
)

def data_processing(data, data_type="train"):
  mfcc = []
  for tensor, target in data:
    mfcc += train_audio_transforms(tensor)
  return mfcc, targets


In [ ]:
class SpeechCommandSubDataset(Dataset):
    
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels
        # self.label_dict = list_dir
        # self.transform = transform
            
    def __len__(self):
        return len(self.data)    
    
    def __getitem__(self,idx):
        return self.data[idx], self.labels[idx]

In [ ]:
from torch.utils.data import DataLoader,random_split,Dataset

valid_dataset = SpeechCommandSubDataset(tensors, targets)
traindata, testdata = random_split(valid_dataset, [round(len(valid_dataset)*.8), round(len(valid_dataset)*.2)])


trainloader = DataLoader(traindata, batch_size=5, shuffle=True)
testloader = DataLoader(testdata, batch_size=5, shuffle=True)

In [ ]:
#do i need to reshape my data
for I, _data in enumerate(trainloader):
  #waveform is tensor torch.Size([5, 1, 16000])
  #labels is tesnor torch.Size([3]). why 3 and not 5?
  #waveform, labels=  _data 
  print(_data)


Streaming output truncated to the last 5000 lines.

        [[ 0.0020,  0.0023,  0.0019,  ..., -0.0049, -0.0048, -0.0038]],

        [[ 0.0050,  0.0052,  0.0057,  ..., -0.0064, -0.0063, -0.0060]],

        [[ 0.0021, -0.0024, -0.0112,  ...,  0.0121,  0.0125,  0.0064]],

        [[ 0.0001,  0.0005,  0.0008,  ..., -0.0019, -0.0016, -0.0015]]]), tensor([1, 1, 1, 0, 0])]
[tensor([[[ 3.2654e-03,  9.4604e-04, -1.4954e-03,  ...,  4.3640e-03,
           3.4485e-03,  5.6763e-03]],

        [[ 2.1362e-04,  3.3569e-04,  4.2725e-04,  ...,  6.1035e-04,
           6.1035e-04,  8.2397e-04]],

        [[ 2.7466e-04,  6.1035e-05,  2.4414e-04,  ..., -7.0190e-04,
          -1.1902e-03, -3.9673e-04]],

        [[-2.1362e-03, -2.8687e-03, -3.9673e-03,  ...,  1.4343e-03,
           5.2185e-03,  6.2256e-03]],

        [[-3.0518e-05, -6.1035e-05, -6.1035e-05,  ...,  3.0518e-05,
           0.0000e+00, -1.2207e-04]]]), tensor([0, 0, 0, 0, 0])]
[tensor([[[ 3.0518e-05, -4.5776e-04, -1.2207e-04,  ...,  1.6479e-03,

In [ ]:
class learningWithoutForgetting(object):
    def __init__(self, pretrained_model_path, optimizer = 'Adam', num_epochs = 10):
        self.num_epochs = num_epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # load model
        print("Loading the pre-trained model...")
        self.model = models.cnnModel()
        # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        self.model.load_state_dict(copy.deepcopy(torch.load(pretrained_model_path, map_location='cpu')), strict=False)
        # Loss function and optimizer
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)

    def evaluate_model(self, X_test, y_test):
        """ Evaluate the model's performance on the test set X_test and y_test

        Parameters:
        ----------
        X_test: torch tensor of MFCC
        y_test: torch tensor of labels

        Returns:
        -------
        accuracy: average accuracy of the model
        accuracies: array of accuracy for each class
        """
        accuracy = 0  # average accuracy of the model
        accuracies = np.zeros(10)  # accuracy for each class
        nb_occurences = np.zeros(10)  # to convert counts to accuracy

        with torch.no_grad():
            for i in range(len(X_test)):
                prediction = self.model.predict(X_test[i].unsqueeze(0).to(device=self.device, dtype=torch.float))
                label = (int)(y_test[i].to(device= self.device).item())
                if prediction == label:
                    accuracies[label] = accuracies[label] + 1
                    accuracy = accuracy + 1
                nb_occurences[label] = nb_occurences[label] + 1

        accuracy = accuracy / len(X_test)
        accuracies = np.divide(accuracies, nb_occurences)

        return accuracy, accuracies

    def cl_train(self, inputs, labels):
        best_accuracy = 0.0
        print("Started adaptation of a model !")
        zipped = zip(inputs, labels)
        for i, (input, label) in enumerate(zipped):
            input = input.unsqueeze(1).float().to(self.device)
            label = label.unsqueeze(0).long().to(self.device)
            for epoch in range(self.num_epochs):
                self.model.train()
                running_loss = 0.0
                self.optimizer.zero_grad()
                outputs = self.model(input)
                loss = F.cross_entropy(outputs, label)
                loss.backward()
                self.optimizer.step()

    def test(self, inputs, labels):
        self.accuracy, self.accuracies = self.evaluate_model(inputs, labels)
        return

In [ ]:
FILE_PATH = "./savedmodels/best_model_state.pt"
new_training_samples_path = "./dataset/data/sample_input_audio.pt"
batch = torch.load(new_training_samples_path)
inputs = torch.cat(batch, dim=0)
digits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
labels = torch.tensor(digits)

# test for different outputs:
meta_dataloader = MetaAudioDataLoader(10, 0.2)
test_inputs, test_labels = meta_dataloader.get_test_samples(5)
# Analysis for Adam tranfer learning
tl_adam = learningWithoutForgetting(pretrained_model_path=FILE_PATH, num_epochs=2)
tl_adam.cl_train(inputs, labels)
tl_adam.test(test_inputs, test_labels)

print(tl_adam.accuracy)

/content/drive/MyDrive/Colab Notebooks/git_projects/AudioClassificationWithDeepLearningAnalysis/dataset/Meta_Dataloader.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  inputs = torch.tensor(inputs)


Loading the pre-trained model...
Started adaptation of a model !
0.1


In [ ]:
import torch
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

NameError: ignored

In [ ]:
labels

['backward',
 'bed',
 'bird',
 'cat',
 'dog',
 'down',
 'eight',
 'five',
 'follow',
 'forward',
 'four',
 'go',
 'happy',
 'house',
 'learn',
 'left',
 'marvin',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'sheila',
 'six',
 'stop',
 'three',
 'tree',
 'two',
 'up',
 'visual',
 'wow',
 'yes',
 'zero']